In [1]:
import pandas as pd 

In [2]:
file_path = r"C:\MyDocs\Machine learning\Club Mahindra\train_5CLrC8b\train.csv"
df = pd.read_csv(file_path)

In [ ]:
df.shape

## starting Exploratory data analysis and noting down things to concentrate on

In [ ]:
df.describe()

In [3]:
import sys
sys.path.append("C:\MyDocs\Machine learning")

In [4]:
import ml_utils

In [5]:
ml_utils.get_columns_with_null_values(df)

season_holidayed_code has 114 null values
state_code_residence has 4764 null values


## There is no way to predict state_code_residence i.e. Resident state of member hence skipping the column from analysis.

## As for the column season_holidayed_code , we can take median of the dataset and fill it in place of nulls

In [ ]:
cmdf = df.drop(columns=['state_code_residence'])

In [ ]:
cmdf.shape

In [ ]:
cmdf['season_holidayed_code'].value_counts()

In [ ]:
cmdf['season_holidayed_code'] = cmdf['season_holidayed_code'].fillna(2.0)

In [ ]:
cmdf.shape

In [ ]:
ml_utils.get_columns_with_null_values(cmdf)

In [ ]:
df['state_code_residence'].value_counts()

## Randomly alloting state id might be possible but not sure if that will be helpful

## Let's explore relations between features and target graphically one by one

## Need to transform column checkin_date to month and year so as look relation year wise and month wise

In [ ]:
import time


In [ ]:
cmdf['Month'] = list(map(lambda x: time.strptime(x, "%d/%m/%y").tm_mon, cmdf['checkin_date']))


In [ ]:
cmdf['Year'] = list(map(lambda x: time.strptime(x, "%d/%m/%y").tm_year, cmdf['checkin_date']))

In [ ]:
cmdf.drop(columns=['checkin_date'], inplace=True)

In [ ]:
cmdf.shape

In [ ]:
cmdf[['Month', 'amount_spent_per_room_night_scaled']].groupby('Month').mean()

In [ ]:
cmdf[['Month', 'amount_spent_per_room_night_scaled']].groupby('Month').sum()

In [ ]:
cmdf[['Month', 'amount_spent_per_room_night_scaled']].groupby('Month').std()

In [ ]:
cmdf[['Month', 'amount_spent_per_room_night_scaled']].groupby('Month').describe()

In [ ]:
%matplotlib inline 

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.style.use('ggplot') # optional: for ggplot-like style

# check for latest version of Matplotlib
print ('Matplotlib version: ', mpl.__version__) # >= 2.0.0


In [ ]:
cmdf[['Month', 'amount_spent_per_room_night_scaled']].groupby('Month')['amount_spent_per_room_night_scaled'].plot(figsize=(20,8))

In [ ]:
grouped = cmdf[['Month', 'amount_spent_per_room_night_scaled']].groupby('Month')

grouped.plot(kind= 'box', figsize=(20,8))
plt.show()

In [ ]:
cmdf.boxplot(column='amount_spent_per_room_night_scaled', by='Month', figsize=(20,10))
plt.ylabel('Amount Spent')
plt.show()


## This signifies that across Months the amount spent is similar i.e. there is no spike in amount spent in a paticular month.

In [ ]:
cmdf.boxplot(column='amount_spent_per_room_night_scaled', by='Year', figsize=(20,10))
plt.ylabel('Amount Spent')
plt.show()

## In years 2015,2016 2017, 2018 amount spent is increasing. For 2019 there seems a dip but that could be because 2019's data is less. 

In [ ]:
cmdf.boxplot(column='amount_spent_per_room_night_scaled', by='main_product_code', figsize=(20,10))
plt.ylabel('Amount Spent')
plt.show()

## product code 1 seems to have highest contribution to amount spent while product code 4 seems to have lowest contribution
## Let's confirm this with other graphs

In [ ]:
cmdf[['main_product_code', 'amount_spent_per_room_night_scaled']].plot(kind='hist')

In [ ]:
cmdf[['main_product_code', 'amount_spent_per_room_night_scaled']].plot(kind='hist', x='main_product_code', y='amount_spent_per_room_night_scaled')

In [ ]:
cmdf[['main_product_code', 'amount_spent_per_room_night_scaled']].plot()

In [ ]:
cmdf[['main_product_code', 'amount_spent_per_room_night_scaled']].head()

In [ ]:
cmdf[['main_product_code', 'amount_spent_per_room_night_scaled']].plot(kind='area', stacked=False)

## Area plot is not the right plot to get any understanding here

In [ ]:
import numpy as np
count, binedges = np.histogram(cmdf['amount_spent_per_room_night_scaled'])

In [ ]:
binedges

In [ ]:
prodrev = cmdf[['main_product_code', 'amount_spent_per_room_night_scaled']]

In [ ]:
byproducttype = prodrev.groupby('main_product_code').sum()

In [ ]:
byproducttype.plot(kind='pie', 
                    figsize=(5, 6),
                    autopct='%1.1f%%', # add in percentages
                    startangle=90,     # start angle 90° (Africa)
                    shadow=True,       # add shadow      
                   subplots=True)
plt.title('Amount spend per room night')

## since its amount spend per room night this is not the correct. Let's find out total spend and then plot a pie

In [ ]:
cmdf['Total'] = cmdf['amount_spent_per_room_night_scaled']*cmdf['roomnights']

In [ ]:
prodrev = cmdf[['main_product_code', 'Total']]

In [ ]:
byproducttype = prodrev.groupby('main_product_code').sum()

In [ ]:
byproducttype.plot(kind='pie', 
                    figsize=(5, 6),
                    autopct='%1.1f%%', # add in percentages
                    startangle=90,     # start angle 90° (Africa)
                    shadow=True,       # add shadow      
                   subplots=True)
plt.title('Amount spent by product code')

In [ ]:
monthlyrev = cmdf[['Month', 'Total']]
bymonth = monthlyrev.groupby('Month').sum()
bymonth.plot(kind='pie',
                    figsize=(5, 6),
                    autopct='%1.1f%%', # add in percentages
                    startangle=90,     # start angle 90° (Africa)
                    shadow=True,       # add shadow      
                   subplots=True)
plt.title('Amount spent Monthly')

In [ ]:
monthlyrev = cmdf[['Year', 'Total']]
byyear = monthlyrev.groupby('Year').sum()
byyear.plot(kind='pie',
                    figsize=(5, 6),
                    autopct='%1.1f%%', # add in percentages
                    startangle=90,     # start angle 90° (Africa)
                    shadow=True,       # add shadow      
                   subplots=True)
plt.title('Amount spent Yearly')

In [ ]:
monthlyrev = cmdf[['channel_code', 'Total']]
bychannel = monthlyrev.groupby('channel_code').sum()
bychannel.plot(kind='pie',
                    figsize=(5, 6),
                    autopct='%1.1f%%', # add in percentages
                    startangle=90,     # start angle 90° (Africa)
                    shadow=True,       # add shadow      
                   subplots=True)
plt.title('Amount spent by Channelcode')

In [ ]:
persons_rev = cmdf[['total_pax', 'Total']]

In [ ]:
for each in cmdf.loc[cmdf.total_pax == 0].index:
    #cmdf['total_pax'][each] = cmdf['numberofadults'][each] + cmdf['numberofchildren'][each]
    print(each)


In [ ]:
cmdf.loc[cmdf.total_pax == 0]

In [ ]:
persons_rev = cmdf[['total_pax', 'Total']]

In [ ]:
persons_rev.plot(kind='hist', y= 'Total')

In [ ]:
persons_rev['total_pax'].value_counts()

In [ ]:
region = cmdf[['resort_region_code', 'Total']]
byregion = region.groupby('resort_region_code').sum()
byregion.plot(kind='pie',
                    figsize=(5, 6),
                    autopct='%1.1f%%', # add in percentages
                    startangle=90,     # start angle 90° (Africa)
                    shadow=True,       # add shadow      
                   subplots=True)
plt.title('Amount spent by RegionCode')

In [ ]:
season = cmdf[['season_holidayed_code', 'Total']]
byseason = season.groupby('season_holidayed_code').sum()
byseason.plot(kind='pie',
                    figsize=(5, 6),
                    autopct='%1.1f%%', # add in percentages
                    startangle=90,     # start angle 90° (Africa)
                    shadow=True,       # add shadow      
                   subplots=True)
plt.title('Amount spent by seasonCode')

In [ ]:
season.boxplot(column='Total', by='season_holidayed_code', figsize=(20,8))

In [ ]:
season = cmdf[['season_holidayed_code', 'amount_spent_per_room_night_scaled']]
season.boxplot(column='amount_spent_per_room_night_scaled', by='season_holidayed_code', figsize=(20,8))

In [ ]:
season.plot(kind='hist', y='amount_spent_per_room_night_scaled')

In [ ]:
season['season_holidayed_code'].value_counts()

In [ ]:
cmdf['state_code_resort'].value_counts()

In [ ]:
stateresorts = cmdf[['state_code_resort', 'Total']]
bystate = stateresorts.groupby('state_code_resort').sum()
bystate.plot(kind='pie',
                    figsize=(5, 6),
                    autopct='%1.1f%%', # add in percentages
                    startangle=90,     # start angle 90° (Africa)
                    shadow=True,       # add shadow      
                   subplots=True)
plt.title('Amount spent by StateCode')

In [ ]:
cmdf['member_age_buckets'].value_counts()

In [ ]:
agewise = cmdf[['member_age_buckets', 'Total']]
byage = agewise.groupby('member_age_buckets').sum()
byage.plot(kind='pie',
                    figsize=(5, 6),
                    autopct='%1.1f%%', # add in percentages
                    startangle=90,     # start angle 90° (Africa)
                    shadow=True,       # add shadow      
                   subplots=True)
plt.title('Amount spent by AgeBucket')

In [ ]:
cmdf['booking_type_code'].value_counts()

In [ ]:
booktype = cmdf[['booking_type_code', 'Total']]
bybook = booktype.groupby('booking_type_code').sum()
bybook.plot(kind='pie',
                    figsize=(5, 6),
                    autopct='%1.1f%%', # add in percentages
                    startangle=90,     # start angle 90° (Africa)
                    shadow=True,       # add shadow      
                   subplots=True)
plt.title('Amount spent by BookingType')

In [ ]:
booktype = cmdf[['booking_type_code', 'amount_spent_per_room_night_scaled']]
booktype.boxplot(column= 'amount_spent_per_room_night_scaled', by='booking_type_code')

In [ ]:
cmdf['cluster_code'].value_counts()

In [ ]:
clusterwise =cmdf[['cluster_code', 'Total']]
bycluster = clusterwise.groupby('cluster_code').sum()
bycluster.plot(kind='pie',
                    figsize=(5, 6),
                    autopct='%1.1f%%', # add in percentages
                    startangle=90,     # start angle 90° (Africa)
                    shadow=True,       # add shadow      
                   subplots=True)
plt.title('Amount spent by ClusterCode')

In [ ]:
cmdf['reservationstatusid_code'].value_counts()

## Now trying Regression algorithm on selected columns

In [ ]:
cols = ['channel_code', 'main_product_code', 'resort_region_code', 'roomnights', 'season_holidayed_code',
'state_code_resort', 'total_pax', 'member_age_buckets', 'cluster_code', 'amount_spent_per_room_night_scaled'
]

In [ ]:
data = cmdf[cols]

In [ ]:
data.shape

In [ ]:
Y, X = data['amount_spent_per_room_night_scaled'], data.drop(columns=['amount_spent_per_room_night_scaled'])

In [ ]:
Y.head()

In [ ]:
X.shape

In [ ]:
X.head()

In [ ]:
X['member_age_buckets'].value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder
l = LabelEncoder()
X['member_age_buckets'] = l.fit_transform(X['member_age_buckets'])

In [ ]:
X['member_age_buckets'].value_counts()

In [ ]:
#l.fit_transform(X['cluster_code'])
X['cluster_code'].value_counts()

In [ ]:
X['cluster_code'] = l.fit_transform(X['cluster_code'])

In [ ]:
X['cluster_code'].value_counts()

In [ ]:
X.head()

In [ ]:
X['season_holidayed_code'].value_counts()
#X['season_holidayed_code'].apply(lambda x: int(x))

In [ ]:
X['season_holidayed_code'] = X['season_holidayed_code'].apply(lambda x: int(x))

In [ ]:
X['season_holidayed_code'].value_counts()

In [ ]:
X.head()

In [ ]:
X = X.drop(columns=['state_code_resort'])

In [ ]:
X.head()

In [ ]:
X = X.drop(columns=['cluster_code'])

In [ ]:
X.head()

In [ ]:
X = pd.get_dummies(X, columns= ['channel_code', 'main_product_code', 'resort_region_code', 'season_holidayed_code', 'member_age_buckets'])

In [ ]:
X.head()

In [ ]:
X['amount_spent_per_room_night_scaled'] = Y

In [ ]:
X.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="ticks", color_codes=True)

In [ ]:
sns.catplot(x="channel_code", y="amount_spent_per_room_night_scaled", data=X);

In [ ]:
sns.catplot(x="channel_code", y="amount_spent_per_room_night_scaled", kind='boxen', data=X.sort_values('channel_code'))

In [ ]:
sns.catplot(x="channel_code", y="amount_spent_per_room_night_scaled", kind='boxen', hue='resort_region_code', data=X.sort_values('channel_code'))

In [ ]:
sns.catplot(x="resort_region_code", y="amount_spent_per_room_night_scaled", kind='boxen', hue='season_holidayed_code', data=X.sort_values('channel_code'))

In [ ]:
print(X.columns)
print(Y.shape)
from sklearn import linear_model
from sklearn.model_selection import train_test_split
regr = linear_model.LinearRegression()
#Y, X = X['amount_spent_per_room_night_scaled'], X.drop(columns=['amount_spent_per_room_night_scaled'])
#X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=.3, random_state=2)


In [ ]:
X_train.shape

In [ ]:
X_train.columns

In [ ]:
X = pd.get_dummies(X, columns= ['channel_code', 'main_product_code', 'resort_region_code', 'season_holidayed_code', 'member_age_buckets'], drop_first=True)

In [ ]:
X.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=.3, random_state=2)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
regr.fit(X_train, y_train)
# The coefficients
print ('Coefficients: ', regr.coef_)
print ('Intercept: ',regr.intercept_)

In [ ]:
cols = ['channel_code', 'main_product_code', 'resort_region_code', 'roomnights', 'season_holidayed_code',
'state_code_resort', 'total_pax', 'member_age_buckets', 'cluster_code', 'amount_spent_per_room_night_scaled'
]

In [ ]:
X, Y = data[['channel_code', 'main_product_code', 'resort_region_code', 'roomnights', 'season_holidayed_code', 'total_pax', 'member_age_buckets']], data['amount_spent_per_room_night_scaled']

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
X.head()

In [ ]:
l = LabelEncoder()
X.loc[:, 'member_age_buckets'] = l.fit_transform(X['member_age_buckets'])


In [ ]:
X.head()

In [ ]:
X.loc[:, 'season_holidayed_code'] = X.loc[:, 'season_holidayed_code'].apply(lambda x: int(x))

In [ ]:
X.head()

In [ ]:
Y.head()

In [ ]:
X = pd.get_dummies(X, columns = ['channel_code', 'main_product_code', 'resort_region_code', 'season_holidayed_code',
'member_age_buckets'], drop_first=True)

In [ ]:
X.head()

In [ ]:
X_train, X_test , y_train, y_test = train_test_split(X,Y, test_size=.2, random_state=2 )

In [ ]:
from sklearn.model_selection import LinearRegression

In [ ]:
from sklearn import linear_model
regr = linear_model.LinearRegression()

In [ ]:
regr.fit(X_train, y_train)

In [ ]:
regr.coef_

In [ ]:
regr.intercept_

In [ ]:
ypred = regr.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
100*mean_squared_error(y_test, ypred)

## Linear Regression produced RMSE*100 as 113.406 

In [ ]:
from sklearn import svm
from sklearn.metrics import mean_squared_error
'''
for each_kernel in ['SVR', 'NuSVR','LinearSVR']:
    svr = getattr(svm, 'SVR')(gamma='auto')
    svr.fit(X_train,y_train)
    ypred= svr.predict(X_test)
    print(each_kernel, 100*mean_squared_error(y_test, ypred))'''

    

In [4]:
def get_process_data():
    X, Y = data[['channel_code', 'main_product_code', 'resort_region_code', 'roomnights', 'season_holidayed_code', 'total_pax', 'member_age_buckets']], data['amount_spent_per_room_night_scaled']
    l = LabelEncoder()
    X.loc[:, 'member_age_buckets'] = l.fit_transform(X['member_age_buckets'])
    X.loc[:, 'season_holidayed_code'] = X.loc[:, 'season_holidayed_code'].apply(lambda x: int(x))
    X = pd.get_dummies(X, columns = ['channel_code', 'main_product_code', 'resort_region_code', 'season_holidayed_code',
    'member_age_buckets'], drop_first=True)
    return X, Y
X, Y = get_process_data()

NameError: name 'data' is not defined

In [2]:
import sys
sys.path.append("C:\MyDocs\Machine learning")
import ml_utils


In [3]:
file_path = r"C:\MyDocs\Machine learning\Club Mahindra\train_5CLrC8b\train.csv"
df = ml_utils.get_data_from_file(file_path)

In [4]:
import ml_utils
X, Y = ml_utils.split_columns_in_x_y(df)

In [4]:
X.head()

,channel_code,main_product_code,resort_region_code,roomnights,season_holidayed_code,total_pax,member_age_buckets,numberofadults,numberofchildren
0,3,1,3,1,2.0,3,F,2,0
1,1,1,3,5,2.0,2,F,2,0
2,1,1,1,4,2.0,2,F,2,0
3,1,1,2,5,2.0,2,F,2,2
4,1,1,2,5,2.0,2,F,2,0


In [5]:
import pandas as pd
df1 = pd.DataFrame({'a': [1,2,3],
                   'b': [4,5,6],
                   'c':[7,8,9]})

In [29]:
df1.loc[df1['a']% 2 == 0].index
#df1['a'] = df1['c']-df1['b']

Int64Index([1], dtype='int64')

In [17]:
df1

,a,b,c
0,3,4,7
1,3,5,8
2,3,6,9


In [30]:
df1

,a,b,c
0,1,4,7
1,2,5,8
2,3,6,9


In [6]:
for each in X.loc[X.total_pax == 0].index:
    X['total_pax'][each] = X['numberofadults'][each] + X['numberofchildren'][each]
    print(each)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


19160
53662
72730
107887
113931
124396
143187
179743
181049
185518
189877
226336
265591
278707
293555
296521


In [32]:
X.loc[X.total_pax == 0]

,channel_code,main_product_code,resort_region_code,roomnights,season_holidayed_code,total_pax,member_age_buckets,numberofadults,numberofchildren


In [33]:
X.head()

,channel_code,main_product_code,resort_region_code,roomnights,season_holidayed_code,total_pax,member_age_buckets,numberofadults,numberofchildren
0,3,1,3,1,2.0,3,F,2,0
1,1,1,3,5,2.0,2,F,2,0
2,1,1,1,4,2.0,2,F,2,0
3,1,1,2,5,2.0,2,F,2,2
4,1,1,2,5,2.0,2,F,2,0


In [7]:
X.drop(columns=['numberofadults', 'numberofchildren'], inplace=True)

In [37]:
X.head()

,channel_code,main_product_code,resort_region_code,roomnights,season_holidayed_code,total_pax,member_age_buckets
0,3,1,3,1,2.0,3,F
1,1,1,3,5,2.0,2,F
2,1,1,1,4,2.0,2,F
3,1,1,2,5,2.0,2,F
4,1,1,2,5,2.0,2,F


In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
l = LabelEncoder()
X['member_age_buckets'] = l.fit_transform(X['member_age_buckets'])


In [43]:
X.head()

,channel_code,main_product_code,resort_region_code,roomnights,season_holidayed_code,total_pax,member_age_buckets
0,3,1,3,1,2.0,3,5
1,1,1,3,5,2.0,2,5
2,1,1,1,4,2.0,2,5
3,1,1,2,5,2.0,2,5
4,1,1,2,5,2.0,2,5


In [44]:
X['season_holidayed_code'] = X['season_holidayed_code'].apply(lambda x: int(x))

ValueError: cannot convert float NaN to integer

In [51]:
X.loc[X['season_holidayed_code'].isna()]

,channel_code,main_product_code,resort_region_code,roomnights,season_holidayed_code,total_pax,member_age_buckets
296,1,1,1,4,NaN,4,2
1081,1,1,1,3,NaN,2,6
21702,1,1,1,4,NaN,4,4
25387,3,3,1,5,NaN,3,3
26818,3,3,1,3,NaN,5,7
28869,1,1,1,4,NaN,3,4
33967,1,1,1,10,NaN,3,2
35670,1,1,1,9,NaN,3,5
45498,3,2,1,8,NaN,3,8
45499,3,2,1,8,NaN,3,8


In [10]:
Y = Y.drop(Y.loc[X['season_holidayed_code'].isna()].index)
X = X.drop(X.loc[X['season_holidayed_code'].isna()].index)

In [68]:
X.shape

(341310, 7)

In [69]:
Y.shape

(341310,)

In [11]:
X['season_holidayed_code'] = X['season_holidayed_code'].apply(lambda x: int(x))

In [71]:
X.head()

,channel_code,main_product_code,resort_region_code,roomnights,season_holidayed_code,total_pax,member_age_buckets
0,3,1,3,1,2,3,5
1,1,1,3,5,2,2,5
2,1,1,1,4,2,2,5
3,1,1,2,5,2,2,5
4,1,1,2,5,2,2,5


In [12]:
X = pd.get_dummies(X, columns=['channel_code', 'main_product_code', 'resort_region_code', 'season_holidayed_code', 'member_age_buckets'], drop_first=True)

In [73]:
X.shape

(341310, 22)

In [13]:
from sklearn import svm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split 
'''
for each_kernel in ['SVR', 'NuSVR','LinearSVR']:
    svr = getattr(svm, 'SVR')(gamma='auto')
    svr.fit(X_train,y_train)
    ypred= svr.predict(X_test)
    print(each_kernel, 100*mean_squared_error(y_test, ypred))'''


"\nfor each_kernel in ['SVR', 'NuSVR','LinearSVR']:\n    svr = getattr(svm, 'SVR')(gamma='auto')\n    svr.fit(X_train,y_train)\n    ypred= svr.predict(X_test)\n    print(each_kernel, 100*mean_squared_error(y_test, ypred))"

In [14]:
X_train,X_test, y_train, y_test = train_test_split(X,Y, test_size=.2)


In [15]:
X_train.shape

(273048, 22)

In [21]:
from sklearn.tree import DecisionTreeRegressor
regr_1 = DecisionTreeRegressor(max_depth=7)
regr_1.fit(X_train,y_train)
ypred = regr_1.predict(X_test)
print(100*mean_squared_error(y_test, ypred))

111.69451788463735


In [25]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=6, random_state=0,n_estimators=1000)
regr.fit(X_train,y_train)
ypred = regr.predict(X_test)
print(100*mean_squared_error(y_test, ypred))

111.6933980546808


## So regular regression algorithms dont yield any good results

In [26]:
import keras

ModuleNotFoundError: No module named 'keras'

In [27]:
!pip install keras

In [ ]:
!pip install tensorflow

In [1]:
python3 -m pip install --upgrade https://storage.googleapis.com/tensorflow/mac/cpu/tensorflow-1.12.0-py3-none-any.whl

SyntaxError: invalid syntax (<ipython-input-1-a69c029a2bf8>, line 1)

In [2]:
!pip install --upgrade tensorflow

^C


In [3]:
import keras

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\ProgramData\Anaconda3\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "C:\ProgramData\Anaconda3\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: Access is denied.


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/errors

for some common reasons an

In [1]:
from keras.callbacks import ModelCheckpoint


Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\ProgramData\Anaconda3\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "C:\ProgramData\Anaconda3\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: Access is denied.


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/errors

for some common reasons an